In [1]:
import torch
import numpy
print(torch.__version__)

1.13.1


In [2]:
print(torch.version.cuda)

11.6


In [3]:
import navec

In [4]:
cuda = torch.device('cuda')

In [5]:
import csv
import sys

csv.field_size_limit(int(sys.maxsize / 100000000000))
texts = []
signs = []
dates = []
sites = []
with open('./signed_stem.csv', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        if row[4] == 'interfax':
            continue
        texts.append(row[3])
        signs.append(row[8])
        dates.append(row[2])
        sites.append(row[4])
signs = signs[1:]
print(texts[1])
print(signs[1])
print(sites[1])
print(dates[1])
word_counts = []
for text in texts:
    words = text.split()
    word_counts.append(len(words))
print(sum(word_counts) / len(word_counts))

«ментовск войны» обошл приквел «игр престолов» пиратск ссылк российск сериал «тайн следствия» «ментовск войны» стал лидер числ пиратск ссылок сет оперед приквел «игр престолов» аналитик объясня эт выход нов сезон «чисткой» друг нелегальн контент кадр сериа «тайн следствия» фот студ «панорама» начал год проект холдинг вгтрк «тайн следствия» «ментовск войны» возглав рейтинг фильм сериал количеств ссылок размещен пиратск ресурс так дан рбк представ компан contentscan занима защит аудиовизуальн контент нелегальн распространен перв квартал эт год аналитик contentscan выяв сет тыс ссылок пиратск ресурс где был доступн просмотр скачиван «тайн следствия» тыс ссылок «ментовск войнами» российск сериал числ нелегальн ссылок обошл приквел «игр престолов» — проект «дом дракона» котор был доступ тыс пиратск сайт финальн эпизод эт сериа стал сам просматрива шо американск кана hbo посл «игр престолов» ноч релиз ег посмотрел млн человек «дом дракона» оказа популярн спецвыпуск «гарр поттера» adv rbc ru 

In [6]:
import numpy as np
from gensim.models import Word2Vec
import time

sentences = [text.split(' ') for text in texts]
start = time.time()
model = Word2Vec(sentences=sentences, vector_size=600, window=5, min_count=2, workers=20)
model.save("word2vec.model")
end = time.time()
print(end - start)
#from navec import Navec
#path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
#navec = Navec.load(path)

C:\ProgramData\anaconda3-64\envs\new_pytorch_env\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


54.09370040893555


In [7]:
vectors = [[model.wv[w] for w in text.split(' ') if w in model.wv] for text in texts[1:]]

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

X = vectors
classes = [-1, 0, 1]
for i in range(len(signs)):
    signs[i] = int(signs[i])
y = torch.zeros(len(vectors), 3, device='cuda:0')
for i in range(len(vectors)):
    y[i][classes.index(signs[i])] = 1

In [9]:
unique_date_and_sign = {(dates[i], signs[i]): i for i in range(len(X))}
indices = set(unique_date_and_sign.values())
Xr = [X[i] for i in range(len(X)) if i in indices]
yr = [y[i] for i in range(len(y)) if i in indices]
sitesr = [sites[i] for i in range(len(sites)) if i in indices]
print(len(Xr))
print(len(yr))

import random

def shuffle(X, y, sites):
    indices = [i for i in range(len(X))]
    random.shuffle(indices)
    X = [X[i] for i in indices]
    y = [y[i] for i in indices]
    sites = [sites[i] for i in indices]
    return (X, y, sites)
X, y, newSites = shuffle(X, y, sites)
Xr, yr, newSitesr = shuffle(Xr, yr, sitesr)

3685
3685


In [10]:
count = int(len(Xr) * 0.7)
val_count = int(len(Xr) * 0.8)
Xtrain, Xval, Xtest = Xr[:count], Xr[count:val_count], Xr[val_count:]
Ytrain, Yval, Ytest = yr[:count], yr[count:val_count], yr[val_count:]

In [11]:
Strain, Sval, Stest = newSitesr[:count], newSitesr[count:val_count], newSitesr[val_count:]

In [113]:
count = int(len(X) * 0.7)
val_count = int(len(X) * 0.8)
Xtrain, Xval, Xtest = X[:count], X[count:val_count], X[val_count:]
Ytrain, Yval, Ytest = y[:count], y[count:val_count], y[val_count:]

In [114]:
Strain, Sval, Stest = newSites[:count], newSites[count:val_count], newSites[val_count:]

In [117]:
#TODO: change lstm, linear, relu to model
def accuracy(Ytrain, Ypred, sites):
    count = 0
    ria = 0
    interfax = 0
    rbc = 0
    percentage_str = ""
    
    for i in range(len(Ytrain)):
        _max = 0
        j_max = 0
        for j in range(len(Ypred[i])):
            if _max < yPred[j]:
                _max = yPred[j]
                j_max = j
        if Ytrain[i][j_max] == 1:
            count += 1
        else:
            if sites[i] == 'ria':
                ria += 1
            if sites[i] == 'interfax':
                interfax += 1
            if sites[i] == 'rbc':
                rbc += 1
    return count / len(Xtrain), ria / len(Xtrain), interfax / len(Xtrain), rbc / len(Xtrain)

def calculate_and_accuracy(Xtrain, Ytrain, sites, lstm, linear, relu):
    count = 0
    ria = 0
    interfax = 0
    rbc = 0
    for i in range(len(Xtrain)):
        hidden = (
            torch.rand(1 * 2, 50, device='cuda:0'),
            torch.rand(1 * 2, 50, device='cuda:0')
        )
        x = Xtrain[i]
        x = np.array(x)
        out, _ = lstm(torch.cuda.FloatTensor(x, device='cuda:0'), hidden)
        out = linear(out[-1])
        out = relu(out)
        yPred = out
        yPred = yPred.reshape(-1)
        _max = 0
        j_max = 0
        for j in range(len(yPred)):
            if _max < yPred[j]:
                _max = yPred[j]
                j_max = j
        if Ytrain[i][j_max] == 1:
            count += 1
        else:
            if sites[i] == 'ria':
                ria += 1
            if sites[i] == 'interfax':
                interfax += 1
            if sites[i] == 'rbc':
                rbc += 1
    return count / len(Xtrain), ria / len(Xtrain), interfax / len(Xtrain), rbc / len(Xtrain)

In [12]:
import sys

lstm = nn.LSTM(input_size=600, hidden_size=50, num_layers=1, bidirectional=True)
lstm.to("cuda:0")
linear = nn.Linear(100, 3)
linear.to("cuda:0")
relu = nn.Softmax(dim=0)
relu.to("cuda:0")
optimizer = optim.AdamW(lstm.parameters(), weight_decay=1e-3, lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

batch_size = 1

lstm.train()
for epoch in range(1000):
    total_loss = 0
    global_start = time.time()
    percentage_str = ""
    batches_time = 0
    Ypred = torch.tensor([])
    for i in range(int(len(Xtrain) / batch_size)):
        start = time.time()
        percentage = i / int(len(Xtrain) / batch_size)
        if len(X[i * batch_size:]) >= batch_size:
            batch = Xtrain[i * batch_size : (i + 1) * batch_size]
        else:
            batch = Xtrain[i * batch_size:]
        optimizer.zero_grad()
        for x in batch:
            hidden = (
                torch.rand(1 * 2, 50, device='cuda:0'),
                torch.rand(1 * 2, 50, device='cuda:0')
            )
            x = np.array(x)
            out, _ = lstm(torch.cuda.FloatTensor(x, device='cuda:0'), hidden)
            out = linear(out[-1])
            out = relu(out)
            yPred = out
            Ypred = torch.cat((Ypred, yPred.detach().cpu().reshape(1, 3)), 0)
            loss = loss_fn(yPred, Ytrain[i]) * 1000 / len(Xtrain)
            total_loss += loss.item()
            loss.backward()
        optimizer.step()
        end = time.time()
        batches_time += end - start
        if batches_time >= 0.2:
            sys.stdout.write(" " * len(percentage_str) + '\r')
            percentage_str = f"Epoch {epoch}: {percentage * 100}%, Time: {end - global_start}s, Loss: {total_loss / (i + 1) / batch_size}"
            sys.stdout.write(percentage_str + '\r')
            sys.stdout.flush()
            batches_time = 0
    total_loss /= len(Xtrain)
    sys.stdout.write(" " * len(percentage_str) + '\r')
    _accuracy, ria, inter, rbc = accuracy(Ytrain, Ypred, Strain)
    val_accuracy, val_ria, val_inter, val_rbc = calculate_and_accuracy(Xval, Yval, Sval, lstm, linear, relu)
    global_end = time.time()
    if epoch % 10 == 0 and epoch != 0:
        torch.save(lstm, f"lstm{epoch}.model")
        torch.save(linear, f"linear{epoch}.model")
    sys.stdout.write(" " * len(percentage_str) + '\r')
    print(f"Epoch {epoch}: Time: {global_end - global_start}s: Loss: {total_loss} Accuracy: {_accuracy}, Validation accuracy: {val_accuracy}")
    #print(f"Ria: {ria}, Interfax: {inter}, Rbc: {rbc}")    
    #print(f"ValRia: {val_ria}, ValInterfax: {val_inter}, ValRbc: {val_rbc}")

NameError: name 'accuracy' is not defined

In [ ]:
accuracy(Xtest, Ytest, Stest, lstm, linear, relu)

In [203]:
torch.save(lstm, f"lstmAdamWLr0_01VecSize600Acc83_4.model")
torch.save(linear, f"linearAdamWLr0_01VecSize600Acc83_4.model")